In [35]:
import pandas as pd
from newspaper import Article
import nltk
import ast
import requests
from tqdm.auto import tqdm
tqdm.pandas()
from bs4 import BeautifulSoup, Comment
import numpy as np

In [36]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [46]:
df = pd.read_csv('RepublicaNepal.csv')

In [38]:
# new = pd.read_csv("RepublicaNepal_articles_continue.csv")

In [39]:
# joined = df[['id', 'created_at', 'urls']].set_index('id').join(new[['id', 'article']].set_index('id')).reset_index()

In [40]:
# joined

,id,created_at,urls,article
0,1376900077713678345,2021-03-30 19:57:14+05:45,https://myrepublica.nagariknetwork.com/news/ne...,"KATHMANDU, March 30: The equity market opened ..."
1,1376899173098745862,2021-03-30 19:53:39+05:45,https://myrepublica.nagariknetwork.com/news/go...,"KATHMANDU, March 30: Expressing strong dissati..."
2,1376885989214355469,2021-03-30 19:01:15+05:45,https://myrepublica.nagariknetwork.com/news/yo...,Pandey accuses Oli of misusing state power and...
3,1376871383347519488,2021-03-30 18:03:13+05:45,https://myrepublica.nagariknetwork.com/news/us...,"KATHMANDU, March 30: Nepal Police's Central In..."
4,1376866695051014146,2021-03-30 17:44:35+05:45,https://myrepublica.nagariknetwork.com/news/pm...,"KATHMANDU, March 30: The government is all set..."
...,...,...,...,...
41221,947658764064243713,2018-01-01 08:25:30+05:45,https://goo.gl/6aTfY9,NaN
41222,947658303290580993,2018-01-01 08:23:40+05:45,https://goo.gl/eLy4Pe,NaN
41223,947655493631188992,2018-01-01 08:12:30+05:45,https://goo.gl/CwpTUq,NaN
41224,947654732121743361,2018-01-01 08:09:29+05:45,https://goo.gl/WpfygX,NaN


In [43]:
# joined = joined[joined.article!="404"]

In [44]:
# joined.to_csv('RepublicaNepal.csv', index=False)

In [47]:
df.shape

(41172, 4)

In [48]:
df.head()

,id,created_at,urls,article
0,1376900077713678345,2021-03-30 19:57:14+05:45,https://myrepublica.nagariknetwork.com/news/ne...,"KATHMANDU, March 30: The equity market opened ..."
1,1376899173098745862,2021-03-30 19:53:39+05:45,https://myrepublica.nagariknetwork.com/news/go...,"KATHMANDU, March 30: Expressing strong dissati..."
2,1376885989214355469,2021-03-30 19:01:15+05:45,https://myrepublica.nagariknetwork.com/news/yo...,Pandey accuses Oli of misusing state power and...
3,1376871383347519488,2021-03-30 18:03:13+05:45,https://myrepublica.nagariknetwork.com/news/us...,"KATHMANDU, March 30: Nepal Police's Central In..."
4,1376866695051014146,2021-03-30 17:44:35+05:45,https://myrepublica.nagariknetwork.com/news/pm...,"KATHMANDU, March 30: The government is all set..."


In [49]:
df.tail()

,id,created_at,urls,article
41167,947658764064243713,2018-01-01 08:25:30+05:45,https://goo.gl/6aTfY9,NaN
41168,947658303290580993,2018-01-01 08:23:40+05:45,https://goo.gl/eLy4Pe,NaN
41169,947655493631188992,2018-01-01 08:12:30+05:45,https://goo.gl/CwpTUq,NaN
41170,947654732121743361,2018-01-01 08:09:29+05:45,https://goo.gl/WpfygX,NaN
41171,947654028317507584,2018-01-01 08:06:41+05:45,https://goo.gl/e47Gfo,NaN


In [51]:
df

,id,created_at,urls,article
0,1376900077713678345,2021-03-30 19:57:14+05:45,https://myrepublica.nagariknetwork.com/news/ne...,"KATHMANDU, March 30: The equity market opened ..."
1,1376899173098745862,2021-03-30 19:53:39+05:45,https://myrepublica.nagariknetwork.com/news/go...,"KATHMANDU, March 30: Expressing strong dissati..."
2,1376885989214355469,2021-03-30 19:01:15+05:45,https://myrepublica.nagariknetwork.com/news/yo...,Pandey accuses Oli of misusing state power and...
3,1376871383347519488,2021-03-30 18:03:13+05:45,https://myrepublica.nagariknetwork.com/news/us...,"KATHMANDU, March 30: Nepal Police's Central In..."
4,1376866695051014146,2021-03-30 17:44:35+05:45,https://myrepublica.nagariknetwork.com/news/pm...,"KATHMANDU, March 30: The government is all set..."
...,...,...,...,...
41167,947658764064243713,2018-01-01 08:25:30+05:45,https://goo.gl/6aTfY9,NaN
41168,947658303290580993,2018-01-01 08:23:40+05:45,https://goo.gl/eLy4Pe,NaN
41169,947655493631188992,2018-01-01 08:12:30+05:45,https://goo.gl/CwpTUq,NaN
41170,947654732121743361,2018-01-01 08:09:29+05:45,https://goo.gl/WpfygX,NaN


## Chunk

In [52]:
def article_ktmpost(row):
    try:
        if (row.article != row.article):
            url = row.urls
            req = requests.get(url, headers)
            soup = BeautifulSoup(req.content, 'html.parser')
            result = soup.find("div", {"class": "col-sm-8"})
            return result.text
        return row.article
    except Exception as e:
        print (str(e))
        return 404
    
def article(row):
    try:
        if (row.article != row.article):
            url = row.urls
            article = Article(url)
            article.download()
            article.parse()
            return article.text
        return row.article
    except Exception as e:
        print (str(e))
        return 404  

In [ ]:
file_path = "RepublicaNepal.csv"
result_file_path = "RepublicaNepal_articles_continue1.csv"

pd.read_csv(file_path, nrows=1).head(0).to_csv(result_file_path)
dataframe = pd.read_csv(file_path, iterator=True, chunksize=1000)

for chunk in dataframe:
   chunk['article'] = chunk.progress_apply(article, axis=1)
   chunk.to_csv(result_file_path, mode='a', header=None)

Article `download()` failed with HTTPConnectionPool(host='uk.news-republic.com', port=80): Max retries exceeded with url: /M/4/1460940 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff0cad30be0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')) on URL https://goo.gl/rXZVC
Article `download()` failed with 404 Client Error: Not Found for url: https://myrepublica.nagariknetwork.com/news/police-arrest-30-for-flagging-off-pm-oli/01-5010277 on URL https://myrepublica.nagariknetwork.com/news/police-arrest-30-for-flagging-off-pm-oli/01-5010277
Article `download()` failed with 404 Client Error: Not Found for url: https://myrepublica.nagariknetwork.com/news/us-concerned-over-peace-in-indo-pacific-region-says-us-admiral-davidson/ on URL https://goo.gl/dMYuSW
Article `download()` failed with 404 Client Error: Not Found for url: https://myrepublica.nagariknetwork.com/news/bibeksheel-sajha-party-split-as-thapa-camp-quits/

Article `download()` failed with 404 Client Error: Not Found for url: https://myrepublica.nagariknetwork.com/news/joshi-no-more-in-constitutional-bench/ on URL https://bit.ly/2BWeGt3
Article `download()` failed with 404 Client Error: Not Found for url: https://myrepublica.nagariknetwork.com/news/massive-fire-engulfs-cotton-factory-in-satungal/ on URL https://is.gd/JPPFnJ
Article `download()` failed with 404 Client Error: Not Found for url: https://myrepublica.nagariknetwork.com/news/high-level-committee-formed-to-probe-na-wide-body-aircraft-purchase-scam/ on URL https://is.gd/BzZHg7
Article `download()` failed with 404 Client Error: Not Found for url: https://myrepublica.nagariknetwork.com/news/rjp-n-lawmaker-resham-chaudhary-sworn-in-amid-protest-with-photos/ on URL https://is.gd/7i03hR
Article `download()` failed with 404 Client Error: Not Found for url: https://myrepublica.nagariknetwork.com/news/govt-employees-should-go-to-assigned-workplace-minister-pandit/ on URL https://is.gd/ah

In [46]:
new = pd.read_csv(result_file_path)

In [50]:
new.shape

(29565, 5)

In [52]:
new[new.article!='404'][['id', 'created_at', 'urls', 'article']].to_csv('KathmanduPost.csv', index=False)